GPT-SW3
https://nlu-lab.notion.site/Prompthandboken-7cd943cc230642f2b8243807731e81ae
  
Fick inbjudan genom att anmäla mig via denna länk: https://docs.google.com/forms/d/e/1FAIpQLSebZv__Me6YUO_kFetDZevwXgSsYSVnXWzG5H68MIwN4XmdtQ/viewform

Borde inte ta långt innan man får mejl, men hör av om det tar för lång tid.

In [ ]:
from huggingface_hub import login  
login()

In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, AutoModelForSequenceClassification
from datasets import load_dataset

# Model name: AI-Sweden-Models/gpt-sw3-135m, AI-Sweden-Models/gpt-sw3-1.3b, AI-Sweden-Models/gpt-sw3-2.7b etc. (ju mer parametrar desto mer RAM & GPU-minne krävs)
model_name = "AI-Sweden-Models/gpt-sw3-1.3b"
device = "cuda:0" if torch.cuda.is_available() else "cpu" #in case i run this on my laptop or pc

In [2]:

# initialize Tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
#model = AutoModelForCausalLM.from_pretrained(model_name, output_hidden_states=True)
# model.eval()
# model.to(device)

Some weights of the model checkpoint at AI-Sweden-Models/gpt-sw3-1.3b were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at AI-Sweden-Models/gpt-sw3-1.3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

dataset = load_dataset("amcoff/skolmat")
print(dataset)

Found cached dataset skolmat (C:/Users/Dasmit/.cache/huggingface/datasets/amcoff___skolmat/default/0.0.0/e974832966434c6f7909c4befc52b70faa93e4095e09e728bfccebcde37bf15f)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['meal', 'label'],
        num_rows: 901
    })
})


In [4]:
def tokenize_function(examples):
    return tokenizer(examples["meal"], padding="max_length", truncation=True)
print(dataset)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["train"]

DatasetDict({
    train: Dataset({
        features: ['meal', 'label'],
        num_rows: 901
    })
})


Map:   0%|          | 0/901 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
import evaluate
import numpy as np
from transformers import DataCollatorForLanguageModeling
def compute_metrics(eval_pred):
    metric = evaluate.load_metric("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False, mlm_probability=0.15
)

In [6]:

training_args = TrainingArguments(output_dir="./results", evaluation_strategy="epoch"
                                  ,num_train_epochs=1,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4
                                  )

# initialize Trainer
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=eval_dataset,         # evaluation dataset
    compute_metrics=compute_metrics
)

trainer.train()


The following columns in the training set don't have a corresponding argument in `GPT2ForSequenceClassification.forward` and have been ignored: meal. If meal are not expected by `GPT2ForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Dasmit\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 901
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 226
  Number of trainable parameters = 1343873024


  0%|          | 0/226 [00:00<?, ?it/s]

ValueError: expected sequence of length 4 at dim 1 (got 17)

In [10]:
def generate_text(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)

    output = model.generate(
        inputs = input_ids,
        max_new_tokens = 200,
        do_sample=True,
        top_p=1, temperature=0.7
        )
    return tokenizer.decode(output[0])

prompt = "Följande är en konversation mellan en besökare på museet och en guide. Guiden arbetar på museet. Guiden är hjälpsam, informativ och mycket vänlig. \n" \
            "Museet innehåller tre utställningar, den första utställningen heter 'Hitta nemo igen' av Hermann Gustafsson \n\n\n"\
            "Besökare: Hej, jag är här för att se utställningen 'Hitta nemo igen', och jag har en fråga!\n" \
            "Guide: Hej, vad kul! Vad vill du veta om utställningen?\n" \
            "Besökare: Vet du vem som skapade utställningen?\n" 
print(generate_text(prompt))         

Följande är en konversation mellan en besökare på museet och en guide. Guiden arbetar på museet. Guiden är hjälpsam, informativ och mycket vänlig. 
Museet innehåller tre utställningar, den första utställningen heter 'Hitta nemo igen' av Hermann Gustafsson 


Besökare: Hej, jag är här för att se utställningen 'Hitta nemo igen', och jag har en fråga!
Guide: Hej, vad kul! Vad vill du veta om utställningen?
Besökare: Vet du vem som skapade utställningen?
Guide: Ja, det är Hermann Gustafsson, hans son.
Besökare: Är ni släkt?
Guide: Nej, det är inte vi.
Besökare: Ok, jag förstår.


Besökare: Hej, jag är intresserad av din familj. Hur många barn har ni?
Guide: Hej! Jag har tre barn, två flickor och en pojke.
Besökare: Ok, är dina barn här?
Guide: Ja, de bor hos sin mamma.
Besökare: Ok, ska vi gå till det här rummet?
Guide: Ja, det tycker jag.
Besökare: Ok, jag har en fråga till!
Guide: Ok, vad vill du veta om utställningen?
Besökare: Vem uppfann den?
Guide: Hermann Gustafsson, hans son.
Besök